In [1]:
from leeq.utils.ai.experiment_generation.experiment_generation import generate_experiment
from leeq.utils.ai.experiment_generation.pulse_sequences import generate_pulse_sequences


In [2]:
description = """
Please measure the T1 time of the qubit in the following way:
1. Prepare the qubit to the excited state by applying a pi pulse.
2. Delay a time t, sweep time t 
3. Measure the qubit after the decay time.
4. fit the qubit excited state population versus time t.
5. Plot the excited state population versus time t both the data points and the fitted results. 

"""

In [3]:
generate_experiment(description)

Experiment Generation Progress:   0%|          | 0/5 [00:00<?, ?it/s]

import numpy as np
import matplotlib.pyplot as plt
from qiskit_experiments.library import DelayPrimitive, basic
from qiskit_experiments.framework import BaseExperiment
from qiskit_experiments.curve_analysis import fit_exp_decay_with_cov, fit_exp_decay


class T1Experiment(BaseExperiment):
    def __init__(self):
        super().__init__()
        self.data = None
        self.fit_params = None
        self.time_step = None

    def run(self, dut_qubit, t, collection_name='f01', mprim_index=0):
        c1 = dut_qubit.get_c1(collection_name)
        pi_pulse = c1['X'].clone()
        pi_pulse.update_pulse_args(amp=1.0, phase=0.0, shape='square', width=1.0)
        delay = DelayPrimitive({'delay_time': t})
        mprim = dut_qubit.get_measurement_prim_intlist(mprim_index)
        lpb = pi_pulse + delay + mprim
        basic(lpb, None, '<z>')
        self.data = np.squeeze(mprim.result())
        self.fit_params = self.fit_exponential_decay(self.data, time_step=t)
        return None

   

'```python\nfrom labchronicle import log_and_record, register_browser_function\nfrom leeq import Experiment\nfrom leeq.core.primitives.logical_primitives import LogicalPrimitiveBlock\nfrom leeq.utils.compatibility import prims\nfrom leeq import basic_run as basic\nimport numpy as np\nimport matplotlib.pyplot as plt\n\nclass T1Experiment(Experiment):\n\n    @log_and_record\n    @register_browser_function()\n    def run(self,\n            dut_qubit: Any,\n            t: float,\n            collection_name: str = \'f01\',\n            mprim_index: int = 0) -> None:\n        """\n        Run the T1 experiment on a qubit and fit the excited state population versus time.\n\n        Parameters:\n        dut_qubit (Any): Device under test (DUT) qubit object.\n        t (float): Delay time.\n        collection_name (str): Collection name for retrieving c1. Default is \'f01\'.\n        mprim_index (int): Index for retrieving measurement primitive. Default is 0.\n        """\n        c1 = dut_qub

In [4]:
description = """

Please measure the T2 echo time of the qubit in the following way:
1. Prepare the qubit to the super position state by applying a pi/2 pulse.
2. Delay a time t, sweep time t 
3. apply a X gate to flip the qubit
4. Delay a same amount of time t
5. applying a pi/2 pulse.
6. Measure the qubit after the decay time.
7. fit the qubit excited state population versus time t.
8. Plot the excited state population versus time t both the data points and the fitted results. 

"""

In [5]:
generate_experiment(description)

Experiment Generation Progress:   0%|          | 0/5 [00:00<?, ?it/s]

from typing import Any
import numpy as np
from qiskit.pulse.primitives import DelayPrimitive
from qiskit_experiments.framework import Experiment
from qiskit_experiments.library.characterization import basic
from qiskit_experiments.library.fitting_models import fit_exp_decay_with_cov

class T2EchoExperiment(Experiment):
    def __init__(self):
        super().__init__()
        self.data = None
        self.fit_params = None
        self.T2_echo_time = None

    def run(self, dut_qubit: Any, t: float, collection_name: str = 'f01', mprim_index: int = 0):
        c1 = dut_qubit.get_c1(collection_name)
        pi_half_pulse = c1['Yp'].clone()
        x_gate = c1['X'].clone()
        delay_time = {'delay_time': t}
        delay = DelayPrimitive(delay_time)
        mprim = dut_qubit.get_measurement_prim_intlist(mprim_index)
        lpb = pi_half_pulse + delay + x_gate + delay + pi_half_pulse + mprim
        basic(lpb, swp, '<z>')
        self.data = np.squeeze(mprim.result())
        self.fi

"```python\nfrom typing import Any\nimport numpy as np\nfrom qiskit.pulse.primitives import DelayPrimitive\nfrom qiskit_experiments.framework import Experiment\nfrom qiskit_experiments.library.characterization import basic\nfrom qiskit_experiments.library.fitting_models import fit_exp_decay_with_cov\n\n@log_and_record\n@register_browser_function()\nclass T2EchoExperiment(Experiment):\n    def run(self, dut_qubit: Any, t: float, collection_name: str = 'f01', mprim_index: int = 0) -> None:\n        '''\n        Run the T2 echo experiment to measure the echo time of a qubit.\n\n        Parameters:\n        dut_qubit (Any): Device under test (DUT) qubit object.\n        t (float): Delay time for the experiment.\n        collection_name (str): Collection name for retrieving c1. Default is 'f01'.\n        mprim_index (int): Index for retrieving measurement primitive. Default is 0.\n        '''\n        c1 = dut_qubit.get_c1(collection_name)\n        pi_half_pulse = c1['Yp'].clone()\n        